In [1]:
import re
import pandas as pd
import pymysql
import openai
import warnings
warnings.filterwarnings('ignore')

def openapi(prompt):
    openai.api_key = OPEN_API
    completion = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          temperature=0,
          max_tokens=2000,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0,
          messages=[{"role": "user", "content": f"{prompt}"}]
    )
    respones_text = completion.choices[0].message.content
    respones_text = respones_text.replace("\n\n", "")
    return respones_text

def en_prompt_len(question_text, input_text):
    prompt = "".join([input_text, question_text])
    if len(prompt) >= 2000:
        input_text = input_text[:2000-len(question_text)]
        prompt = "".join([input_text, question_text]) 
    return prompt

def description_en(number):
    try:
        store_id = number
        lang = 'en'

        conn = pymysql.connect(host='db-6j3k3.pub-cdb.ntruss.com', db = "redtable2021", user='redtable', password='fpemxpdlqmf5491!@#', cursorclass = pymysql.cursors.DictCursor)
        curs = conn.cursor()
        sql = """
            SELECT s.id AS 'store_id', 
                    if(st.name IS NOT NULL AND st.name != "", st.name, "-" ) AS '매장명',
                    if(st.branch_name IS NOT NULL AND st.branch_name != "", st.branch_name, "-") AS '지점명',
                    if(slt.name IS NOT NULL AND slt.name !="", slt.name, "-") AS '카테고리',
                    if(st.holiday IS NOT NULL AND st.holiday !="", st.holiday, "-") AS '정규휴무일', 
                    if(subway.subway_walk IS NOT NULL AND subway.subway_walk != "", CONCAT(subway.subway_walk, 'min away from the ', subway.subway_name), if(subway.subway_name IS NOT NULL AND subway.subway_name !="", subway.subway_name, "-")) AS '가까운 지하철역',
                    if(price.avg_price IS NOT NULL AND price.avg_price != "-", price.avg_price, "-") AS '가격대',
                    if(GROUP_CONCAT(DISTINCT p.name SEPARATOR ' | ') IS NOT NULL AND GROUP_CONCAT(DISTINCT p.name SEPARATOR ' | ') != "", GROUP_CONCAT(DISTINCT p.name SEPARATOR ' | '), "-") AS '대표메뉴',
                    if(st.description IS NOT NULL AND st.description != "", st.description, "-") AS '매장소개',
                    if(st.address_road IS NOT NULL AND st.address_road != "", SUBSTRING_INDEX(st.address_road, ", ", -1), if(st.address IS NOT NULL AND st.address != "", SUBSTRING_INDEX(st.address, ", ", -1), "")) AS 'AREA1'
            FROM store s
            LEFT JOIN store_translate st ON s.id = st.store_id AND st.lang = '{1}'
            LEFT JOIN store_label sl ON s.id = sl.store_id
            LEFT JOIN store_label_master slm ON sl.store_label_master_id = slm.id
            LEFT JOIN store_label_translate slt ON slm.id = slt.store_label_master_id AND slt.lang = '{1}'
            LEFT JOIN (SELECT s.id, lt.name AS 'subway_name', CAST(sld.distance AS signed INTEGER) AS 'subway_distance', ROUND((sld.distance * 60) / 4000, 0) AS 'subway_walk', ROW_NUMBER() OVER(PARTITION BY sld.store_id ORDER BY sld.distance ASC) AS distance_rank
                       FROM store s
                       INNER JOIN store_landmark sld ON s.id = sld.store_id
                       INNER JOIN landmark_master lm ON sld.landmark_master_id = lm.id AND lm.type = "subway_station"
                       INNER JOIN landmark_translate lt ON lm.id = lt.landmark_master_id AND lt.lang = '{1}'
                       WHERE sld.store_id = {0}) subway ON s.id = subway.id AND subway.distance_rank = 1
            LEFT JOIN (SELECT p.id, p.store_id, pt.name AS 'name', ROW_NUMBER() OVER (PARTITION BY p.store_id ORDER BY sorting DESC) sorting_rank
                       FROM product p
                       LEFT JOIN product_translate pt ON p.id = pt.product_id AND pt.lang = '{1}'
                       WHERE p.is_main = 1 AND p.status = 'normal' AND p.name != '식사이용권' AND (p.name LIKE '%+%' OR p.name NOT LIKE '%금액권%') AND p.store_id = {0}) p ON p.store_id = s.id AND p.sorting_rank <= 3
            LEFT JOIN (SELECT store_id, if(CAST(AVG(price) / 10000 AS signed INTEGER) > 0, CONCAT(CAST(AVG(price) / 10000 AS signed INTEGER), "0,000 KRW"), "-") AS 'avg_price'
                       FROM product
                       WHERE is_main = 1 AND status = 'normal' AND name != '식사이용권' AND (name LIKE '%+%' OR name NOT LIKE '%금액권%') AND price >= 1000 AND store_id = {0}
                       GROUP BY store_id) price ON price.store_id = s.id
            WHERE s.id = {0}
            GROUP BY s.id
        """
        curs.execute(sql.format(store_id, lang))
        result = curs.fetchall()
        df_store = pd.DataFrame(result)

        sql = """
            SELECT p.store_id, pt.name AS '상품명', 
                if(GROUP_CONCAT(DISTINCT pct.name SEPARATOR ',') IS NOT NULL AND GROUP_CONCAT(DISTINCT pct.name SEPARATOR ',') != "", GROUP_CONCAT(DISTINCT pct.name SEPARATOR ','), "-") AS '상품라벨',
                if(CAST(p.price AS signed INTEGER) > 1000, CAST(p.price AS signed INTEGER), "-") AS '가격',
                if(pt.description IS NOT NULL AND pt.description != "", pt.description, "-") AS '상품정보',
                if(pt.includes IS NOT NULL AND pt.includes != "", pt.includes, "-") AS '포함정보'
            FROM product p
            LEFT JOIN product_translate pt ON p.id = pt.product_id AND pt.lang = '{1}'
            LEFT JOIN product_category pc ON p.id = pc.product_id
            LEFT JOIN product_category_master pcm ON pc.product_category_master_id = pcm.id
            LEFT JOIN product_category_translate pct ON pcm.id = pct.product_category_master_id AND pct.lang = '{1}'
            WHERE p.status = 'normal' AND pt.name IS NOT NULL AND pt.name != "" AND p.is_main = 1 AND p.name != '식사이용권' AND p.store_id = {0} AND (p.name LIKE '%+%' OR p.name NOT LIKE '%금액권%')
            GROUP BY p.id
            ORDER BY p.sorting DESC
            LIMIT 3;
        """
        curs.execute(sql.format(store_id, lang))
        result = curs.fetchall()
        df_product = pd.DataFrame(result)
        conn.close()
        curs.close()

        pattern = r'\([^)]*\)'
        line = "" + "\n"
        product_text = ""
        if len(df_store["대표메뉴"]) > 0:
            for i in range(len(df_store["store_id"])):
                df_store["대표메뉴"][i] = re.sub(pattern=pattern, repl='', string=df_store["대표메뉴"][i])

        if len(df_product) > 0:
            for i in range(len(df_product["store_id"])):
                df_product["상품명"][i] = re.sub(pattern=pattern, repl='', string=df_product["상품명"][i])

            for i in range(len(df_product["상품명"])):    
                product_sn = "Product List : Product" + str(len(df_product["상품명"])-i) + "\n"
                product_name = "Product Name : " + df_product["상품명"][i] + "\n"
                product_label = "Product Category : " + df_product["상품라벨"][i] + "\n"
                #product_price = "Product Price : " + df_product["가격"][i] + " KRW" + "\n"
                product_description = "Product Information : " + df_product["상품정보"][i] + "\n"
                product_includes = "Included : " + df_product["포함정보"][i] + "\n"
                product_text = "".join([product_sn, product_name, product_label, product_description, product_includes, product_text, line])

        question_text = "Generate a creative phrase about the store with the given information.\n"
        store_name = "Store Name : " + df_store["매장명"][0] + "\n"
        branch_name = "Branch Name : " + df_store["지점명"][0] + "\n"
        area1 = "Area : " + df_store["AREA1"][0] + "\n"
        category_name = "Category : " + df_store["카테고리"][0] + "\n"
        #holiday_information = "Holiday Information : " + df_store["정규휴무일"][0] + "\n"
        #nearby_subway = "Nearby Subway Station : " + df_store["가까운 지하철역"][0] + "\n"
        #price = "Average Price : " + df_store["가격대"][0] + "\n"
        representative_menu = "Representative Menu : " + df_store["대표메뉴"][0] + "\n"
        #store_introduction = "Store Introduction : " + df_store["매장소개"][0] + "\n"
        product_list = product_text
        input_text = "".join([store_name, branch_name, area1, category_name, representative_menu, product_list, line])
        
        prompt = en_prompt_len(question_text, input_text)
        respones_text = openapi(prompt)

        # Area 표출시 잘못된 지역 교체하기
        if not df_store["매장명"][0].lower() in "seoul":
            respones_text = respones_text.replace("Seoul",df_store["AREA1"][0])

        return respones_text
    except:
        print("Store ID : " + str(number) + " 가 실행되지 않았습니다.")

In [2]:
description_en(948573)

"Welcome to Artist's Table, a hidden gem in Gyeongsangnam-do where culinary artistry meets delectable flavors. Indulge in our signature Brunch Set Menu, savor the aromatic Bugoksik Soup Curry, or experience the exquisite taste of Yeonghonui Hankki. Each dish is carefully crafted to ignite your senses and leave you craving for more. Step into our world of gastronomic masterpieces and let your taste buds dance with delight."